# Data importation

In [84]:
# Import modules
import numpy as np
import pandas as pd
import scipy
from scipy import ndimage as ndi
import seaborn as sns
import re
import warnings
import matplotlib.pyplot as plt
from skimage.measure import label
import sklearn
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# https://stackoverflow.com/questions/19970764/making-feature-vector-from-gabor-filters-for-classification
# https://stackoverflow.com/questions/20608458/gabor-feature-extraction


In [85]:
TF = 'C7'
df = pd.read_csv ('../Features/TF1_'+TF+'.csv',index_col = 0)
ypet_intensity = df.copy()
ypet_intensity

,patch,img,TF_name,blur_lapl,blur_ski,patch_size,mean_intensity,sum_intensity,median_intensity,standard_deviation,...,gabro_15,lbp_0,lbp_1,lbp_2,lbp_3,lbp_4,lbp_5,lbp_6,lbp_7,Circularity
0,patch_6,1,C - 7,False,False,557,790.666068,440401,790.0,56.209692,...,0.006944,0.259459,0.167568,0.094595,0.102703,0.064865,0.083784,0.089189,0.137838,0.726082
1,patch_7,1,C - 7,False,False,739,1042.802436,770631,926.0,274.833903,...,0.046875,0.208835,0.178715,0.102410,0.128514,0.076305,0.078313,0.076305,0.150602,0.794946
2,patch_8,1,C - 7,False,False,680,841.535294,572244,831.0,89.429541,...,0.064338,0.247140,0.178490,0.091533,0.112128,0.089245,0.082380,0.064073,0.135011,0.853092
3,patch_9,1,C - 7,False,False,1234,870.455429,1074142,845.0,117.994204,...,0.004808,0.200000,0.201220,0.097561,0.100000,0.091463,0.080488,0.069512,0.159756,0.812795
4,patch_11,1,C - 7,False,False,1106,828.970163,916841,829.0,56.703426,...,0.019817,0.234890,0.192308,0.093407,0.081044,0.057692,0.086538,0.076923,0.177198,0.880464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5496,patch_505,12,C - 7,False,False,1298,828.140216,1074926,819.5,76.862345,...,0.023585,0.234524,0.176190,0.102381,0.100000,0.044048,0.075000,0.080952,0.186905,0.658896
5497,patch_506,12,C - 7,False,False,821,2108.556638,1731125,2109.0,157.649052,...,0.007353,0.221612,0.148352,0.120879,0.104396,0.087912,0.089744,0.089744,0.137363,0.908437
5498,patch_508,12,C - 7,False,False,1372,790.250000,1084223,757.0,130.311276,...,0.037500,0.219163,0.188326,0.105727,0.078194,0.075991,0.085903,0.085903,0.160793,0.596698
5499,patch_509,12,C - 7,False,False,1088,881.883272,959489,867.0,105.353520,...,0.034226,0.243243,0.194879,0.089616,0.072546,0.049787,0.079659,0.096728,0.173542,0.889439


# Preprocessing

## Parameters to adapt for the data processing

In [86]:
#parameters to adapt for analyzation:
#outlier removal
outlier = True              #If outliers in general should be removed True, if not False
interquantile = True        #if outliers should be removed by defining a range with the interquantile: True; to remove outliers just when they're exceeding a certain quantile: False
quantile_keep = 0.95        #to change if interquantile = False; else ignore
outlier_columns = ['mean_intensity'] #choose columns of which the outliers should be removed. to choose all the columns: list(ypet_intensity_processed)[6:]
outlier_range = 1.5         #if interquantile = True defined this value to define the range. Gets multiplicated with interquantile. Else ignore
#Normalization
normalization = True
columns_to_be_normalized =  ['mean_intensity','median_intensity','sum_intensity'] #list(ypet_intensity_processed)[6:]  #if all columns should be normalized
                            #take logs to reduce skewness
take_log = False             #want to take the log of certain columns
columns_log = ['mean_intensity'] #needs pre-analyzation of columns if log is needed. With a histogram it can be seen if the distribution is skewed. 
ypet_intensity_processed = ypet_intensity.copy()


## Outlier removal in columns of the dataframe

In [87]:
def outlier_removal(without_outliers, **kwargs):
    #define input parameters
    outlier_range =   kwargs['outlier_range']
    outlier_columns = kwargs['outlier_columns'] 
    interquantile =   kwargs['interquantile']
    quantile_keep =   kwargs['interquantile']
    # first choose which method for outlier removal wants to be used. Use a for loop to iterate over the defined columns of the dataframe.
    if interquantile == True:
        for col in outlier_columns:
            # defined the interquantile range and then the upper and lower limit of the choosen column
            median = without_outliers[col].median()
            q_75 = without_outliers[col].quantile(q = 0.75)
            q_25 = without_outliers[col].quantile(q = 0.25)
            interquantile = q_75 - q_25                              
            upper_bound = median + (interquantile * outlier_range)
            lower_bound = median - (interquantile * outlier_range)

            # Create a boolean mask that is True for rows with a value less than or equal to the upper limit and higher or equal to the lower limit
            mask = (without_outliers[col] <= upper_bound) & (without_outliers[col] >= lower_bound)
            # Use the mask to filter the dataframe
            without_outliers = without_outliers[mask]
    else: 
        for col in columns_outliers:
            #define the limit up to which the values are kept
            quantile_limit = without_outliers[col].quantile(q = quantile_keep)
            #define a boolean mask that is True for rows that are in the defined limit
            mask = without_outliers[col] <= quantile_limit
            #use the mask to filter the dataframe
            without_outliers = without_outliers[mask]
        
    return without_outliers


In [88]:
#apply the function for outlier removal
if outlier == True:
    #group by the TF and just cut the outliers for one TF
    without_outliers = ypet_intensity_processed.groupby('TF_name',as_index=False).apply(outlier_removal, outlier_range=outlier_range, outlier_columns=outlier_columns, interquantile = interquantile, quantile_keep=quantile_keep).reset_index()
    #change index so dataframe is as before
    without_outliers = without_outliers.drop(['level_0'],axis=1)
    without_outliers = without_outliers.set_index('level_1')
    without_outliers.index.name = None
    #save it under a new dataframe
    ypet_intensity_processed = without_outliers.copy()

## Take logarithm of certain columns to get rid of the skewness

In [89]:
#take logs of certain columns that are skewed
if take_log == True:
    TF_grouped = ypet_intensity_processed.groupby('TF_name')
    log_rows = TF_grouped[columns_log].transform(lambda x: np.log(x))
    ypet_intensity_processed[columns_log] = log_rows 

## Normalization of columns of the dataframe

In [90]:
# Normalization of columns of the dataframe
if normalization == True:
    TF_grouped = ypet_intensity_processed.groupby('TF_name')
    Normalized_columns = TF_grouped[columns_to_be_normalized].transform(lambda x: (x - x.mean()) / x.std()) #removed rows_to_be to have all col normalized

    # Replace normalized
    ypet_intensity_processed[columns_to_be_normalized] = Normalized_columns                         


# Task 1

In [91]:
print(list(ypet_intensity_processed))

['patch', 'img', 'TF_name', 'blur_lapl', 'blur_ski', 'patch_size', 'mean_intensity', 'sum_intensity', 'median_intensity', 'standard_deviation', 'variance', 'skewness', 'kurtosis', 'interquartile_range ', 'entropy', 'Perimeter', 'area_Test', 'axis_major_length', 'feret_diameter_max', 'axis_minor_length', 'solidity', 'similarity', 'hull area', 'correlation coef', 'gabro_0', 'gabro_1', 'gabro_2', 'gabro_3', 'gabro_4', 'gabro_5', 'gabro_6', 'gabro_7', 'gabro_8', 'gabro_9', 'gabro_10', 'gabro_11', 'gabro_12', 'gabro_13', 'gabro_14', 'gabro_15', 'lbp_0', 'lbp_1', 'lbp_2', 'lbp_3', 'lbp_4', 'lbp_5', 'lbp_6', 'lbp_7', 'Circularity']


# Data exploration

## Scatterplots of different parameters compared to each other

Scatterplots (if you want with regression line (degree 1))

In [92]:
mk_size = 0.01 #Choose a small mk_size if a lot of datapoints, so densities are easier to read
points_col = 'black'
line_col   = 'blue'

plt.figure()
plt.subplot(221)
sns.regplot(y="patch_size", x="mean_intensity", data=ypet_intensity_processed,marker = '.',scatter_kws={'s':mk_size}, color = points_col, fit_reg=False)
plt.subplot(222)
sns.regplot(y="patch_size", x="median_intensity", data=ypet_intensity_processed,marker = '.',scatter_kws={'s':mk_size}, color = points_col, fit_reg=False)
plt.subplot(223)
sns.regplot(y="patch_size", x="sum_intensity", data=ypet_intensity_processed,marker = '.',scatter_kws={'s':mk_size}, color = points_col, fit_reg=False)
plt.subplot(224)
sns.regplot(y="patch_size", x="correlation coef", data=ypet_intensity_processed,marker = '.',scatter_kws={'s':mk_size}, color = points_col, fit_reg=False)
plt.axvline(np.mean(ypet_intensity_processed['correlation coef']),color = line_col,linewidth = 0.5,label = "Mean")
plt.legend()
plt.suptitle("Densities of various measure for TF "+TF)
plt.tight_layout()
plt.savefig("densities_"+TF=".svg")
plt.show()

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (4292180342.py, line 18)

## Parameters to define

In [ ]:
maxdegree = 3                          #To define is the maximum degree of the polynomial that should be tried (maxdegree of 5 takes degree up to 4 into account)
x_features = ['mean_intensity','sum_intensity','correlation coef','median_intensity'] #define a list with all the columns that could be interesting for task 1
decide_x_features = np.array([2])      #set the list of indexes for the X values used for regression from list defined in test_features
decide_y_feature = 'patch_size'        #also other features could be taken into account


## Prepare test and train sets and X and y

In [ ]:
#Define features used with the right format, following x_features and decide_x_features
features_used = [x_features[i] for i in decide_x_features]

# Split in train and test set
df_train, df_test = train_test_split(ypet_intensity_processed, train_size = 0.8, test_size = 0.2, random_state = 10)

#define the x and the y for the test and train sets
df_train_x = df_train[features_used]
df_test_x = df_test[features_used]

df_train_y = df_train[[decide_y_feature]]
df_test_y = df_test[[decide_y_feature]]

## Apply cross validation

In [ ]:
#model to select
cross_validation_ridge_error = np.zeros(maxdegree)
cross_validation_lm_error = np.zeros(maxdegree)

#see which degree fits data the best for linear regression
for d in range(1, maxdegree+1):    #it will create 1,2...maxdregree range vector like this
    #polynomial feature expansion of x_train
    x_poly_train = PolynomialFeatures(degree=d).fit_transform(df_train_x)
    #apply linear regression model and cross-validation for alpha-ridge regression
    lr = LinearRegression()
    rr = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
    #apply cross validation
    cve = cross_validate(lr,x_poly_train,df_train_y,scoring='neg_mean_squared_error', cv=5, return_train_score=True)
    crr = cross_validate(rr,x_poly_train,df_train_y,scoring='neg_mean_squared_error', cv=5, return_train_score=True)
    #make array for cross validation with linear model and for ridge regression
    cross_validation_lm_error[d-1] = np.mean(np.absolute(cve['test_score']))
    cross_validation_ridge_error[d-1] = np.mean(np.absolute(crr['test_score']))

## Choose best model

In [ ]:
index_min_lm = np.argmin(cross_validation_lm_error)
index_min_ridge = np.argmin(cross_validation_ridge_error)

poly_lm_test = PolynomialFeatures(degree=index_min_lm+1)
x_poly_test_lm = poly_lm_test.fit_transform(df_test_x)
poly_lm_train = PolynomialFeatures(degree=index_min_lm+1)
x_poly_train_lm = poly_lm_train.fit_transform(df_train_x)

x_poly_train_df_lm = pd.DataFrame(x_poly_train_lm)
x_poly_test_df_lm = pd.DataFrame(x_poly_test_lm)

poly_ridge_test = PolynomialFeatures(degree=index_min_ridge+1)
x_poly_test_ridge = poly_ridge_test.fit_transform(df_test_x)
poly_ridge_train = PolynomialFeatures(degree=index_min_ridge+1)
x_poly_train_ridge = poly_ridge_train.fit_transform(df_train_x)

x_poly_train_df_ridge = pd.DataFrame(x_poly_train_ridge)
x_poly_test_df_ridge = pd.DataFrame(x_poly_test_ridge)

#make linear model
model_lm = LinearRegression().fit(x_poly_train_df_lm, df_train_y)
model_ridge = RidgeCV(alphas=[1e-4,1e-3, 1e-2, 1e-1, 1]).fit(x_poly_train_df_ridge, df_train_y)

#train error
y_train_pred_lm = model_lm.predict(x_poly_train_lm)
mse_train_lm = mean_squared_error(df_train_y,y_train_pred_lm)
y_train_pred_ridge = model_ridge.predict(x_poly_train_ridge)
mse_train_ridge = mean_squared_error(df_train_y,y_train_pred_ridge)

#test error
y_test_pred_lm = model_lm.predict(x_poly_test_df_lm)
mse_test_lm = mean_squared_error(df_test_y,y_test_pred_lm)
y_test_pred_ridge = model_ridge.predict(x_poly_test_df_ridge)
mse_test_ridge = mean_squared_error(df_test_y,y_test_pred_ridge)

## Visualize the best model and give back the parameters

In [ ]:
df_plot_sort = pd.DataFrame(df_test_x)
df_plot_sort['reg'] =  y_test_pred_lm[:,0]
df_plot_sort.sort_values(features_used[0],inplace = True)

In [ ]:
#show predicted points with linear model

if (len(features_used)!=1):
    warnings.warn("WARNING features_used should be of len 1 for this plot")
else:
    plt.figure()
    plt.plot(df_train_x,df_train_y, 'r.',markersize = 1,label = "Train set")
    plt.plot(df_plot_sort[features_used[0]],df_plot_sort['reg'],color = 'black',label = "Regression on test set")
    plt.ylabel("Nucleus size")
    plt.xlabel(features_used[0])    #WARNING features_used should be of len 1 for this plot
    plt.title("Linear model")
    plt.legend()
    #plt.yscale("log")
    #plt.xscale("log")
    plt.show()


''' OLD WAY TO PLOT IT (if need to go back)
#show predicted points with linear model
plt.plot(df_train_x,df_train_y, 'ro')
plt.plot(df_test_x,y_test_pred_lm, 'o',color = 'black')
#plt.yscale("log")
#plt.xscale("log")
plt.show()
'''

In [ ]:
#show predicted points with linear model
df_plot_sort_ridge = pd.DataFrame(df_test_x)
df_plot_sort_ridge['reg'] =  y_test_pred_ridge[:,0]
df_plot_sort_ridge.sort_values(features_used[0],inplace = True)

if (len(features_used)!=1):
    warnings.warn("WARNING features_used should be of len 1 for this plot")
else:
    plt.figure()
    plt.plot(df_train_x,df_train_y, 'r.',markersize = 1,label = "Train set")
    plt.plot(df_plot_sort_ridge[features_used[0]],df_plot_sort_ridge['reg'],color = 'black',label = "Regression on test set")
    plt.ylabel("Nucleus size")
    plt.xlabel(features_used[0])    #WARNING features_used should be of len 1 for this plot
    plt.title("Ridge regression model")
    plt.legend()
    #plt.yscale("log")
    #plt.xscale("log")
    plt.show()

''' OLD WAY TO PLOT IT (if need to go back)
#show predicted points with ridge regression
plt.plot(df_train_x,df_train_y, 'ro')
plt.plot(df_test_x,y_test_pred_ridge, 'o',color = 'black')
#plt.yscale("log")
#plt.xscale("log")
plt.show()
'''


## Output of the coefficients 

In [ ]:
if mse_test_ridge >= mse_test_lm:
    feature_names = poly_lm_test.get_feature_names_out(input_features=features_used)
    print('take linear regression')
    print('features: ',feature_names)
    print('coefficients: ',model_lm.coef_)
    print('intercept: ',model_lm.intercept_)
    print('R^2-score: ', r2_score(df_train_y,y_train_pred_lm))
    

else:
    feature_names = poly_ridge_test.get_feature_names_out(input_features=features_used)
    print('take ridge regression')
    print('features: ',feature_names)
    print('coefficients: ',model_ridge.coef_)
    print('intercept: ',model_ridge.intercept_)
    print('alpha: ',model_ridge.alpha_)
    print('R^2-score: ', r2_score(df_train_y,y_train_pred_ridge))